<a href="https://colab.research.google.com/github/acram002/AI-Driven-Recipe-Suggestion-System/blob/main/flanSmallModelTester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import AutoTokenizer

from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/My Drive/full_dataset.csv"
df = pd.read_csv(file_path)  # Load dataset
df.head(3)  # Display first few rows

Mounted at /content/drive


,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."


In [2]:
import json
# Function to convert a row to ShareGPT-style format
def format_to_sharegpt(row):
    ingredients = ", ".join(eval(row["ingredients"]))  # Join list of ingredients into a string
    directions = "\n".join(f"{i+1}. {step}" for i, step in enumerate(eval(row["directions"])))  # Format steps
    return {
        "conversations": [
            {
                "from": "human",
                "value": f"Generate a recipe using the following ingredients: {ingredients}"
            },
            {
                "from": "gpt",
                "value": f"Title: {row['title']}\n\nInstructions:\n{directions}"
            }
        ]
    }

# Apply formatting to each row
formatted_data = [format_to_sharegpt(row) for _, row in df.iterrows()]

# Save to JSON file
with open("sharegpt_recipes.json", "w", encoding="utf-8") as f:
    json.dump(formatted_data, f, indent=2, ensure_ascii=False)

In [3]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define source and destination paths
local_path = "sharegpt_recipes.json"
drive_path = "/content/drive/My Drive/sharegpt_recipes.json"  # You can change the folder path if needed

# Copy the file to Google Drive
shutil.copy(local_path, drive_path)

print(f"✅ File successfully copied to: {drive_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ File successfully copied to: /content/drive/My Drive/sharegpt_recipes.json


In [7]:
pip install transformers datasets peft bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 846.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 834.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!pip install ijson


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 3.4 MB/s eta 0:00:00


In [2]:
import ijson
import json

input_path = "/content/drive/My Drive/sharegpt_recipes.json"
output_path = "/content/drive/My Drive/sharegpt_recipes_small.json"

# Stream the large JSON file and collect only the first 100 records
items = []
with open(input_path, 'r', encoding='utf-8') as f:
    parser = ijson.items(f, "item")
    for i, item in enumerate(parser):
        if i >= 100:
            break
        items.append(item)

# Save trimmed subset
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(items, f, indent=2, ensure_ascii=False)

print(f"✅ Trimmed file saved to: {output_path}")


✅ Trimmed file saved to: /content/drive/My Drive/sharegpt_recipes_small.json


In [4]:
from huggingface_hub import login

login()


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Convert each conversation to plain prompt text
def format_convo(example):
    conversation = example["conversations"]
    messages = ""
    for turn in conversation:
        if turn["from"] == "human":
            messages += f"<|user|>\n{turn['value']}\n"
        elif turn["from"] == "gpt":
            messages += f"<|assistant|>\n{turn['value']}\n"
    return {"text": messages}

# Apply formatting
dataset = dataset.map(format_convo)

# Tokenize dataset
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=dataset.column_names)


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./mistral-recipe-model",
    per_device_train_batch_size=1,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
    learning_rate=2e-5,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none",  # disables wandb/huggingface hub tracking
    push_to_hub=False,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


In [ ]:
trainer.save_model("./mistral-recipe-model")
tokenizer.save_pretrained("./mistral-recipe-model")


In [7]:
# STEP 1: Install dependencies
!pip install -q transformers datasets accelerate

# STEP 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# STEP 3: Load the dataset
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/full_dataset.csv')

# STEP 4: Clean and reduce dataset
df = df[['title', 'ingredients', 'directions']].dropna()
df = df.sample(n=10000, random_state=42)

# STEP 5: Prepare data in prompt-response format
df['prompt'] = 'Generate a recipe:\nTitle: ' + df['title'] + '\nIngredients: ' + df['ingredients']
df['response'] = df['directions']

# STEP 6: Convert to HuggingFace Dataset
from datasets import Dataset

dataset = Dataset.from_pandas(df[['prompt', 'response']])

# STEP 7: Tokenize the data
from transformers import AutoTokenizer

model_name = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    return tokenizer(
        example['prompt'],
        text_target=example['response'],
        truncation=True,
        padding='max_length',
        max_length=512
    )

tokenized_dataset = dataset.map(tokenize, batched=True)

# STEP 8: Load the model
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# STEP 9: Training setup
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# STEP 10: Train the model
trainer.train()
# Grab an actual training example to test generation
sample_prompt = df.iloc[0]['prompt']
sample_target = df.iloc[0]['response']

inputs = tokenizer(sample_prompt, return_tensors="pt", truncation=True).to(model.device)
output = model.generate(**inputs, max_length=512)
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

print("\n=== Sample Prompt from Training ===")
print(sample_prompt)
print("\n=== Target Response ===")
print(sample_target)
print("\n=== Model Generated ===")
print(decoded_output)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

<ipython-input-7-ebc18699483a>:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,3.195800
200,1.030100
300,0.989000
400,0.871700
500,0.788600
600,0.936600
700,0.871400
800,0.809200
900,0.780700
1000,0.824500



=== Sample Prompt from Training ===
Generate a recipe:
Title: Ice Cream Krispies
Ingredients: ["12 cup butter", "1 cup brown sugar", "6 cups crisp rice cereal", "1 cup coconut", "12 cup chopped nuts", "12 gallon vanilla ice cream, softened", "16 ounces strawberries, cleaned, stemmed, & sliced", "1 pint strawberry, cleaned and stemmed", "13 cup white sugar", "1 teaspoon vanilla"]

=== Target Response ===
["Prepare sauce: Cut strawberries in half.", "In a saucepan over medium high heat, combine strawberries, sugar and vanilla.", "Cook, stirring occasionally, until sauce thickens, about 5 minutes.", "Remove from heat.", "In a blender, puree sauce.", "Store sauce in refrigerator until ready to use.", "In large bowl, combine cereal, coconut and chopped nuts.", "Set aside.", "Combine butter and brown sugar in saucepan and warm until smooth.", "Pour over crispy rice mixture and stir.", "Place half of mixture on bottom of 9x13\" pan and level.", "Smooth softened ice cream over top.", "Pour an

In [11]:
# STEP 1: Install dependencies
!pip install -q transformers datasets accelerate

# STEP 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# STEP 3: Load and prep dataset
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/full_dataset.csv')
df = df[['title', 'ingredients', 'directions']].dropna()
df = df.sample(n=5000, random_state=42)  # Reduced for faster training

# Format prompt and response
df['prompt'] = df.apply(lambda row: f"Generate a recipe:\nTitle: {row['title']}\nIngredients: {row['ingredients']}", axis=1)
df['response'] = df['directions']

from datasets import Dataset
dataset = Dataset.from_pandas(df[['prompt', 'response']])

# STEP 4: Tokenize
from transformers import AutoTokenizer

model_name = 'google/flan-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    inputs = tokenizer(
        example['prompt'],
        truncation=True,
        padding='max_length',
        max_length=512
    )
    targets = tokenizer(
        example['response'],
        truncation=True,
        padding='max_length',
        max_length=512
    )

    # Set -100 for pad tokens so loss ignores them
    labels = targets['input_ids']
    labels = [[token if token != tokenizer.pad_token_id else -100 for token in label] for label in labels]

    inputs['labels'] = labels
    return inputs

tokenized_dataset = dataset.map(tokenize, batched=True)

# Sanity check
sample = tokenized_dataset[0]
print("📌 Sample Prompt:\n", tokenizer.decode(sample['input_ids'], skip_special_tokens=True))
print("📌 Sample Target:\n", tokenizer.decode([t for t in sample['labels'] if t != -100]))

# STEP 5: Load model
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# STEP 6: Training args
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/flan_recipe_model',
    per_device_train_batch_size=2,
    num_train_epochs=1,              # Only 1 epoch for speed
    max_steps=500,                   # Hard cap on total steps
    logging_steps=50,
    save_strategy='no',             # Don’t waste time saving multiple checkpoints
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# STEP 7: Train
trainer.train()

# STEP 8: Save model
###output_path = "/content/drive/MyDrive/flan_recipe_model/final"
#model.save_pretrained(output_path)
#tokenizer.save_pretrained(output_path)
print(f"✅ Model saved to {output_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

📌 Sample Prompt:
 Generate a recipe: Title: Ice Cream Krispies Ingredients: ["12 cup butter", "1 cup brown sugar", "6 cups crisp rice cereal", "1 cup coconut", "12 cup chopped nuts", "12 gallon vanilla ice cream, softened", "16 ounces strawberries, cleaned, stemmed, & sliced", "1 pint strawberry, cleaned and stemmed", "13 cup white sugar", "1 teaspoon vanilla"]
📌 Sample Target:
 ["Prepare sauce: Cut strawberries in half.", "In a saucepan over medium high heat, combine strawberries, sugar and vanilla.", "Cook, stirring occasionally, until sauce thickens, about 5 minutes.", "Remove from heat.", "In a blender, puree sauce.", "Store sauce in refrigerator until ready to use.", "In large bowl, combine cereal, coconut and chopped nuts.", "Set aside.", "Combine butter and brown sugar in saucepan and warm until smooth.", "Pour over crispy rice mixture and stir.", "Place half of mixture on bottom of 9x13<unk>" pan and level.", "Smooth softened ice cream over top.", "Pour and level remaining mixt

<ipython-input-11-954cb18714b8>:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.000000
100,0.000000


KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load your saved fine-tuned model
model_path = "/content/drive/MyDrive/flan_recipe_model/final"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Prompt user and generate recipe
user_ingredients = input("Enter your ingredients (comma-separated):\n").split(",")
formatted_ingredients = [f'"{ingredient.strip()}"' for ingredient in user_ingredients]
ingredients_list = "[" + ", ".join(formatted_ingredients) + "]"

prompt = f"Generate a recipe:\nTitle: Custom Dish\nIngredients: {ingredients_list}"
inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)
output = model.generate(**inputs, max_length=512, temperature=0.9, do_sample=True, top_p=0.95)
print("\n=== Generated Recipe ===\n")
print(tokenizer.decode(output[0], skip_special_tokens=True))


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the original pretrained model (not your finetuned one)
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
tokenizer = AutoTokenizer.from_pretrained("t5-small")

prompt = "Generate a recipe:\nTitle: Custom Dish\nIngredients: chicken, rice, broccoli, garlic, soy sauce"

inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)
output = model.generate(**inputs, max_length=512, do_sample=True)
print(tokenizer.decode(output[0], skip_special_tokens=True))


In [2]:
# STEP 11: Save model to a local folder
output_path = "/content/drive/MyDrive/finetuned_recipe_model"
model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

print(f"Model and tokenizer saved to: {output_path}")


Model and tokenizer saved to: /content/drive/MyDrive/finetuned_recipe_model


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# STEP 1: Load model and tokenizer from Google Drive
model_path = "/content/drive/MyDrive/finetuned_recipe_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

def generate_recipe(ingredients):
    prompt = f"Generate a recipe:\nTitle: Custom Dish\nIngredients: {ingredients}"
    print("\n=== Prompt Sent to Model ===\n")
    print(prompt)

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)

    # Try sampling with temperature
    output = model.generate(
        **inputs,
        max_length=512,
        num_beams=4,
        temperature=0.9,
        do_sample=True,
        top_k=50,
        top_p=0.95
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded


user_ingredients = input("Enter your ingredients (comma-separated):\n")
recipe = generate_recipe(user_ingredients)
print("\n=== Generated Recipe Instructions ===\n")
print(recipe)


Enter your ingredients (comma-separated):
chicken, rice, broccoli, garlic, soy sauce

=== Prompt Sent to Model ===

Generate a recipe:
Title: Custom Dish
Ingredients: chicken, rice, broccoli, garlic, soy sauce

=== Generated Recipe Instructions ===


